<a href="https://colab.research.google.com/github/vdslab/t-taiki0620/blob/main/tfidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install unidic-lite
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"
!pip install mojimoji

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl libio-string

In [3]:
import os
import json
import csv
import pandas as pd
import MeCab
import subprocess
import mojimoji

path = '/content/drive/MyDrive/Colab Notebooks/pre-judgit/data.json'

with open(path, encoding='utf8')as f:
  df = pd.read_json(f)

In [4]:
df_2018 = df.iloc[0:0]
print(df_2018)
cnt = 0
for i in range(len(df)):
  if df.loc[i]['公開年度'] == 2018:
    df_2018.loc[cnt] = df.loc[i]
    cnt += 1

Empty DataFrame
Columns: [ID, プロジェクトID, 公開年度, 主要政策・施策, 府省庁, 事業名, 事業の目的, 事業概要, 執行額, objective_x, objective_y]
Index: []


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [5]:
import re
def normalize_text(text):
  # 全部全角に変換
  result = mojimoji.han_to_zen(text)

  # 全角から半角に変換（カナは除く）
  result = mojimoji.zen_to_han(result, kana=False)

  # 全ての文字を小文字に変換
  result = result.lower()
  
  return result
  # return re.sub(' ', '', mojimoji.zen_to_han(text)).upper().split()

In [6]:
# 0404 used this
cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')
def tokenize(text):
  m=MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")
  lines = m.parse(text).splitlines()
  words = []
  for line in lines:
    chunks = line.split('\t')
    # print(chunks[0])
    if chunks[0] != 'EOS' and not chunks[1].startswith('記号'):
      words.append(chunks[0])
  return words

In [7]:
from gensim.models import word2vec
model   = word2vec.Word2Vec.load("/content/drive/MyDrive/Colab Notebooks/pre-judgit/ja-MeCab-50.data.model")

In [8]:
#分かち書きする文
separation = "事業の目的"
#separation = "事業概要"

In [9]:
tf_business_list = []
word_list = []
for i in range(len(df_2018)):
  tf_business_list.append(df_2018.loc[i])
  if df_2018.loc[i][separation] != '':
    if normalize_text(df_2018.loc[i][separation]) != []:
      words = tokenize(normalize_text(df_2018.loc[i][separation]))
      for word in words:
        if word not in word_list:
          word_list.append(word)

In [10]:
tf = {}
for data in tf_business_list:
  tf[data.loc["プロジェクトID"]] = {}
  words = tokenize(normalize_text(data.loc[separation]))
  tmp = {}
  for word in words:
    tmp[word] = words.count(word)/len(words)
  tf[data.loc["プロジェクトID"]] = tmp

In [11]:
import math
tmp = {}
for data in tf_business_list:
  words = tokenize(normalize_text(data.loc[separation]))
  check = []
  for word in words:
    if word not in check:
      check.append(word)
      if word not in tmp.keys():
        tmp[word] = 1
      else:
        tmp[word] += 1

In [12]:
idf = {}
for word in word_list:
  idf[word] = math.log2(len(tf_business_list)/tmp[word])

In [13]:
tfidf = {}
count = 0
for data in tf_business_list:
  if data.loc['プロジェクトID'] not in tfidf.keys():
    tfidf[data.loc['プロジェクトID']] = {}
  words = tokenize(normalize_text(data.loc[separation]))
  ex = {}
  # print(count, data.loc['プロジェクトID'], data.loc['事業名'], words)
  count += 1
  for word in words:
    ex[word] = idf[word] * tf[data.loc['プロジェクトID']][word]
  
  # print(ex)
  tfidf[data.loc['プロジェクトID']] = ex

In [14]:
import json
obj = []
for data in tf_business_list:
  print(data.loc['プロジェクトID'])
  tmp_array = []
  tmp_list = sorted(tfidf[data.loc['プロジェクトID']].items(), key=lambda x : x[1], reverse=True)
  for i in range(5):
    try:
      tmp_array.append({"word": tmp_list[i][0],
                         "score":float(tmp_list[i][1])})
    except:
      print("skip")
  obj.append({'プロジェクトID':int(data.loc['プロジェクトID']),
              'okapi':tmp_array})

with open('/content/drive/MyDrive/Colab Notebooks/pre-judgit/okapi2018.json', 'w', encoding='utf8')as f:
  json.dump(obj, f, indent=2, ensure_ascii=False)

16
15
52
35
84
62
63
83
12
1690
161
1850
1494
14
1677
1614
1641
1857
1385
1507
4908
1509
4337
815
1212
814
1213
816
907
4473
3182
4471
1740
1562
1564
1565
1781
1786
1560
794
939
1109
1248
1770
4838
8200
179
7614
4379
1559
795
8194
173
1233
990
879
3070
2646
3094
1234
3093
4451
4462
8344
4492
1517
4450
5267
3831
4480
4491
60
4521
4487
5254
8322
8293
4046
391
390
388
389
393
392
992
997
403
404
4622
346
345
1003
342
348
409
410
411
412
6938
343
245
849
347
1499
895
1021
720
750
719
483
1192
510
889
1166
799
1174
6935
394
1185
399
226
509
1190
8063
1186
1191
1188
2120
2082
5072
5148
5253
5149
5163
5282
1502
4518
4557
661
313
2803
5
314
657
4433
2887
2966
3186
659
2800
1449
8309
8294
658
8310
2678
1104
2662
2696
3839
1686
8295
8311
2984
1229
2697
3840
255
1228
2633
8296
8312
7812
7813
440
4551
4659
1215
7857
4418
1420
2242
880
940
833
702
742
1129
870
8071
1232
871
220
7730
4531
5320
5112
5334
5141
5164
431
671
433
105
2881
2370
2879
2883
2307
2369
2258
2308
2365
2228
2827
2931
2295
2375
4

In [15]:
tfidf[3550]

{'における': 0.5325166913221974,
 '中央合同庁舎第5号館': 3.023601908270302,
 '工事': 1.8626378608266208,
 '等': 0.14764547377136422}

In [16]:
len(obj)

4373